In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

In [83]:
data = pd.read_csv('D:\data1.csv')

In [84]:
age_bins = [-np.inf  ,17, 25, 33, 41,  np.inf]
age_labels = ['A', 'B', 'C', 'D', 'E'] 
data.Age = pd.cut(data.Age, bins= age_bins, labels= age_labels, right = False, include_lowest = True)

In [85]:
x = data.iloc[:, 2:]
y_age = data.iloc[:, 1]
y_sex = data.iloc[:, 0]

In [86]:
Labels = np.unique(y_age)
N = {i: y_age[y_age == i].size for i in Labels}

In [87]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_re, y_re = oversample.fit_resample(x, y_age)

In [88]:
sc= MinMaxScaler()
pdata=pd.DataFrame(sc.fit_transform(x_re),columns=x_re.columns,index=x_re.index)

In [89]:
x_new = pdata.iloc[:, :]

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x_new , y_re, test_size=0.2 , random_state=1)

In [91]:
reg = LogisticRegression()
reg.fit(x_train,y_train)
lr_pred=reg.predict(x_test)# # linear reg
print("Classification Report is:\n",classification_report(y_test,lr_pred))

Classification Report is:
               precision    recall  f1-score   support

           B       0.50      0.56      0.53        18
           C       0.28      0.31      0.29        16
           D       0.29      0.10      0.15        20
           E       0.33      0.50      0.40        18

    accuracy                           0.36        72
   macro avg       0.35      0.37      0.34        72
weighted avg       0.35      0.36      0.34        72



In [12]:
RFC = RandomForestClassifier()
n_estimators = [500]
max_features = [ 'auto']
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=RFC, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
best_model = grid_search.fit(x_train, y_train)
rf_pred=best_model.predict(x_test)
print("Classification Report is:\n",classification_report(y_test,rf_pred))
#sns.heatmap(confusion_matrix(y_test,rf_pred))
# 'auto' or 'sqrt' or 'log2'

Classification Report is:
               precision    recall  f1-score   support

           B       0.65      0.83      0.73        18
           C       0.60      0.56      0.58        16
           D       0.38      0.25      0.30        20
           E       0.57      0.67      0.62        18

    accuracy                           0.57        72
   macro avg       0.55      0.58      0.56        72
weighted avg       0.55      0.57      0.55        72



In [13]:
mlp=MLPClassifier(activation='tanh',shuffle=True,random_state=2023)
mlp.fit(x_train , y_train )
y_pred=mlp.predict(x_test)
accuracy_score(y_test,y_pred)
mlp_clf = MLPClassifier(hidden_layer_sizes=(150,100,50),
                        max_iter = 300,activation = 'relu',
                        solver = 'adam')

mlp_clf.fit(x_train, y_train)
print(classification_report(y_test, y_pred))

C:\Users\Marjan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

           B       0.48      0.61      0.54        18
           C       0.27      0.25      0.26        16
           D       0.29      0.10      0.15        20
           E       0.37      0.56      0.44        18

    accuracy                           0.38        72
   macro avg       0.35      0.38      0.35        72
weighted avg       0.35      0.38      0.34        72



C:\Users\Marjan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [92]:
y_re_tst = y_re.to_numpy()
y_re_tst.shape

y_re_tst = y_re_tst.reshape(-1,1)
print(y_re_tst.shape)

encoder = OneHotEncoder()
a =encoder.fit(y_re_tst)

y_encoded = a.transform(y_re_tst).toarray()


(360, 1)


In [140]:
from random import randint
x_train, x_test, y_train, y_test = train_test_split(x_new , y_encoded, test_size=0.2 , random_state=randint(100, 1000))
print(x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape)

(288, 12) (72, 12) (288, 4) (72, 4)


In [141]:
model2 = keras.Sequential()

model2.add(Dense(1024, kernel_initializer ="uniform",activation = "relu", input_dim=12))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(512, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(256, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(128, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(64, kernel_initializer = "uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(32, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(16, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(8, kernel_initializer ="uniform",activation = "softmax"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(4))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

In [142]:
model2.compile(optimizer= "SGD", loss = "MSE", metrics = ["accuracy"])

In [143]:
print("Fit model on training data")
history = model2.fit(x_train, y_train, batch_size=70, epochs=100)

Fit model on training data
Epoch 1/100
5/5 [==============================] - 10s 48ms/step - loss: 1.1497 - accuracy: 0.3333
Epoch 2/100
5/5 [==============================] - 0s 41ms/step - loss: 0.9535 - accuracy: 0.4549
Epoch 3/100
5/5 [==============================] - 0s 40ms/step - loss: 0.8690 - accuracy: 0.4653
Epoch 4/100
5/5 [==============================] - 0s 49ms/step - loss: 0.7795 - accuracy: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 46ms/step - loss: 0.7245 - accuracy: 0.5764
Epoch 6/100
5/5 [==============================] - 0s 50ms/step - loss: 0.7006 - accuracy: 0.5590
Epoch 7/100
5/5 [==============================] - 0s 46ms/step - loss: 0.6047 - accuracy: 0.6146
Epoch 8/100
5/5 [==============================] - 0s 41ms/step - loss: 0.6303 - accuracy: 0.5903
Epoch 9/100
5/5 [==============================] - 0s 40ms/step - loss: 0.5791 - accuracy: 0.5729
Epoch 10/100
5/5 [==============================] - 0s 40ms/step - loss: 0.5115 - accuracy

In [144]:
history = model2.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
9/9 [==============================] - 2s 233ms/step - loss: 0.0734 - accuracy: 0.8368 - val_loss: 0.1707 - val_accuracy: 0.4444
Epoch 2/100
9/9 [==============================] - 0s 50ms/step - loss: 0.0717 - accuracy: 0.8611 - val_loss: 0.1703 - val_accuracy: 0.4444
Epoch 3/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0662 - accuracy: 0.8646 - val_loss: 0.1684 - val_accuracy: 0.4306
Epoch 4/100
9/9 [==============================] - 0s 40ms/step - loss: 0.0667 - accuracy: 0.8472 - val_loss: 0.1674 - val_accuracy: 0.4583
Epoch 5/100
9/9 [==============================] - 0s 39ms/step - loss: 0.0668 - accuracy: 0.8507 - val_loss: 0.1649 - val_accuracy: 0.4861
Epoch 6/100
9/9 [==============================] - 0s 47ms/step - loss: 0.0567 - accuracy: 0.8715 - val_loss: 0.1623 - val_accuracy: 0.4861
Epoch 7/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0638 - accuracy: 0.8681 - val_loss: 0.1599 - val_accuracy: 0.5000
Epoch 8/100
9/9 [==

In [159]:
se_test = model2.evaluate(x_test, y_test)


2/2 [==============================] - 2s 33ms/step - loss: 0.1863 - accuracy: 0.2083


In [160]:
yhat_probs = model2.predict(x_test, verbose=0)
yhat_classes = np.argmax(yhat_probs, axis=1)
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:]
#print(yhat_classes)

yhat_classes = yhat_classes.reshape(-1, 1)
#print(yhat_classes.shape)

ohcc = preprocessing.OneHotEncoder()
outt = ohcc.fit_transform(yhat_classes)

In [147]:
accuracy = accuracy_score(y_test, outt)
print('Accuracy: %f' % accuracy)

Accuracy: 0.625000


In [148]:
print("Classification Report is:\n",classification_report(y_test,outt))

Classification Report is:
               precision    recall  f1-score   support

           0       0.59      0.87      0.70        15
           1       0.50      0.47      0.48        17
           2       0.46      0.38      0.41        16
           3       0.86      0.75      0.80        24

   micro avg       0.62      0.62      0.62        72
   macro avg       0.60      0.62      0.60        72
weighted avg       0.63      0.62      0.62        72
 samples avg       0.62      0.62      0.62        72



In [149]:
from genetic_selection import GeneticSelectionCV

estimator = model2.fit(x_train , y_train , batch_size=70, epochs=100 )
model = GeneticSelectionCV(
    estimator, cv=5, verbose=0,
    scoring="accuracy", max_features=12,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x_new, y_re)
print('Features:', x_new.columns[model.support_])

Epoch 1/100
5/5 [==============================] - 0s 43ms/step - loss: 0.0208 - accuracy: 0.9757
Epoch 2/100
5/5 [==============================] - 0s 44ms/step - loss: 0.0297 - accuracy: 0.9618
Epoch 3/100
5/5 [==============================] - 0s 39ms/step - loss: 0.0287 - accuracy: 0.9479
Epoch 4/100
5/5 [==============================] - 0s 52ms/step - loss: 0.0256 - accuracy: 0.9653
Epoch 5/100
5/5 [==============================] - 0s 49ms/step - loss: 0.0261 - accuracy: 0.9618
Epoch 6/100
5/5 [==============================] - 0s 38ms/step - loss: 0.0221 - accuracy: 0.9792
Epoch 7/100
5/5 [==============================] - 0s 39ms/step - loss: 0.0255 - accuracy: 0.9722
Epoch 8/100
5/5 [==============================] - 0s 39ms/step - loss: 0.0256 - accuracy: 0.9653
Epoch 9/100
5/5 [==============================] - 0s 45ms/step - loss: 0.0344 - accuracy: 0.9410
Epoch 10/100
5/5 [==============================] - 0s 49ms/step - loss: 0.0404 - accuracy: 0.9271
Epoch 11/100
5/5 [=

TypeError: estimator should be an estimator implementing 'fit' method, <keras.callbacks.History object at 0x0000020333D273D0> was passed

In [165]:
sc= MinMaxScaler()
p_data=pd.DataFrame(sc.fit_transform(x),columns=x.columns,index=x.index)
x_ne = p_data.iloc[:, :]

In [166]:
y_r = y_age.to_numpy()
y_r.shape

y_r = y_r.reshape(-1,1)
print(y_r.shape)

enc = OneHotEncoder()
aa =enc.fit(y_r)

y_enc = aa.transform(y_r).toarray()


(239, 1)


In [167]:
from random import randint
x_train, x_test, y_train, y_test = train_test_split(x_ne , y_enc, test_size=0.2 , random_state=randint(100, 1000))
print(x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape)

(191, 12) (48, 12) (191, 4) (48, 4)


In [168]:
model2 = keras.Sequential()

model2.add(Dense(1024, kernel_initializer ="uniform",activation = "relu", input_dim=12))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(512, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(256, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(128, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(64, kernel_initializer = "uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(32, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(16, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(8, kernel_initializer ="uniform",activation = "softmax"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(4))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

In [169]:
model2.compile(optimizer= "SGD", loss = "MSE", metrics = ["accuracy"])

In [170]:
print("Fit model on training data")
history = model2.fit(x_train, y_train, batch_size=70, epochs=100)

Fit model on training data
Epoch 1/100
3/3 [==============================] - 11s 55ms/step - loss: 1.2411 - accuracy: 0.2880
Epoch 2/100
3/3 [==============================] - 0s 43ms/step - loss: 1.1359 - accuracy: 0.3613
Epoch 3/100
3/3 [==============================] - 0s 43ms/step - loss: 0.9466 - accuracy: 0.4346
Epoch 4/100
3/3 [==============================] - 0s 45ms/step - loss: 0.8854 - accuracy: 0.4555
Epoch 5/100
3/3 [==============================] - 0s 41ms/step - loss: 0.7785 - accuracy: 0.5497
Epoch 6/100
3/3 [==============================] - 0s 41ms/step - loss: 0.7234 - accuracy: 0.5759
Epoch 7/100
3/3 [==============================] - 0s 43ms/step - loss: 0.7042 - accuracy: 0.5916
Epoch 8/100
3/3 [==============================] - 0s 47ms/step - loss: 0.5977 - accuracy: 0.6754
Epoch 9/100
3/3 [==============================] - 0s 42ms/step - loss: 0.5940 - accuracy: 0.6702
Epoch 10/100
3/3 [==============================] - 0s 41ms/step - loss: 0.6022 - accuracy

In [171]:
history = model2.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
6/6 [==============================] - 2s 417ms/step - loss: 0.0558 - accuracy: 0.9529 - val_loss: 0.1812 - val_accuracy: 0.4583
Epoch 2/100
6/6 [==============================] - 0s 76ms/step - loss: 0.0714 - accuracy: 0.9162 - val_loss: 0.1812 - val_accuracy: 0.4583
Epoch 3/100
6/6 [==============================] - 0s 70ms/step - loss: 0.0676 - accuracy: 0.9529 - val_loss: 0.1809 - val_accuracy: 0.4583
Epoch 4/100
6/6 [==============================] - 0s 64ms/step - loss: 0.0645 - accuracy: 0.9319 - val_loss: 0.1807 - val_accuracy: 0.4583
Epoch 5/100
6/6 [==============================] - 1s 110ms/step - loss: 0.0633 - accuracy: 0.9319 - val_loss: 0.1806 - val_accuracy: 0.4583
Epoch 6/100
6/6 [==============================] - 0s 68ms/step - loss: 0.0537 - accuracy: 0.9529 - val_loss: 0.1811 - val_accuracy: 0.4583
Epoch 7/100
6/6 [==============================] - 0s 42ms/step - loss: 0.0581 - accuracy: 0.9372 - val_loss: 0.1803 - val_accuracy: 0.4583
Epoch 8/100
6/6 [=

In [172]:
se_test = model2.evaluate(x_test, y_test)


2/2 [==============================] - 0s 11ms/step - loss: 0.2095 - accuracy: 0.3958


In [173]:
yhat_probs = model2.predict(x_test, verbose=0)
yhat_classes = np.argmax(yhat_probs, axis=1)
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:]
#print(yhat_classes)

yhat_classes = yhat_classes.reshape(-1, 1)
#print(yhat_classes.shape)

ohcc = preprocessing.OneHotEncoder()
outt = ohcc.fit_transform(yhat_classes)

In [174]:
print("Classification Report is:\n",classification_report(y_test,outt))

Classification Report is:
               precision    recall  f1-score   support

           0       0.20      0.14      0.17         7
           1       0.27      0.60      0.37        10
           2       0.55      0.50      0.52        22
           3       1.00      0.11      0.20         9

   micro avg       0.40      0.40      0.40        48
   macro avg       0.51      0.34      0.32        48
weighted avg       0.53      0.40      0.38        48
 samples avg       0.40      0.40      0.40        48

